In [1]:
#real data with real Feedback
import numpy as np
import pandas as pd
import gurobipy
fixedItemSet=np.array([[1.00000e+00, 4.21669e-01, 1.10000e-05, 1.09020e-02, 3.09585e-01,
        2.57833e-01],
       [1.00000e+00, 2.95442e-01, 1.40000e-05, 1.35191e-01, 2.92304e-01,
        2.77050e-01],
       [1.00000e+00, 4.38513e-01, 3.00000e-06, 3.07140e-02, 3.84494e-01,
        1.46277e-01],
       [1.00000e+00, 3.31830e-01, 2.20000e-05, 1.99040e-02, 4.40390e-01,
        2.07855e-01],
       [1.00000e+00, 3.06008e-01, 4.50000e-04, 7.70480e-02, 2.30439e-01,
        3.86055e-01],
       [1.00000e+00, 3.13277e-01, 1.25000e-04, 1.84130e-02, 4.10555e-01,
        2.57630e-01],
       [1.00000e+00, 3.06008e-01, 4.50000e-04, 7.70480e-02, 2.30439e-01,
        3.86055e-01],
       [1.00000e+00, 2.49086e-01, 1.00900e-03, 5.14682e-01, 6.77320e-02,
        1.67491e-01],
       [1.00000e+00, 2.77121e-01, 1.31000e-04, 3.81530e-02, 3.35835e-01,
        3.48760e-01],
       [1.00000e+00, 3.06008e-01, 4.50000e-04, 7.70480e-02, 2.30439e-01,
        3.86055e-01],
       [1.00000e+00, 3.75829e-01, 2.50000e-05, 3.30410e-02, 3.49637e-01,
        2.41468e-01],
       [1.00000e+00, 2.64355e-01, 1.20000e-05, 3.73930e-02, 4.20649e-01,
        2.77591e-01],
       [1.00000e+00, 2.64355e-01, 1.20000e-05, 3.73930e-02, 4.20649e-01,
        2.77591e-01],
       [1.00000e+00, 3.06008e-01, 4.50000e-04, 7.70480e-02, 2.30439e-01,
        3.86055e-01],
       [1.00000e+00, 2.87909e-01, 2.50000e-05, 8.98300e-03, 5.11333e-01,
        1.91751e-01],
       [1.00000e+00, 3.81149e-01, 1.29000e-04, 6.00380e-02, 2.69129e-01,
        2.89554e-01],
       [1.00000e+00, 2.97322e-01, 2.50000e-05, 3.49510e-02, 4.13566e-01,
        2.54137e-01],
       [1.00000e+00, 2.11406e-01, 3.60000e-05, 2.77300e-03, 5.69886e-01,
        2.15900e-01],
       [1.00000e+00, 2.97750e-01, 1.30000e-05, 1.16030e-02, 5.12182e-01,
        1.78452e-01],
       [1.00000e+00, 2.81649e-01, 1.73000e-04, 1.95994e-01, 1.51003e-01,
        3.71182e-01]])
userclick=np.load(r'./Yahoo!/userclick.npy',allow_pickle=True)
user=np.argmax([len(userclick[i]) for i in range(len(userclick))])
def hwithoutconstraints(action):
    t=model.iteration
    if type(action)==torch.Tensor:
        action=action.detach().numpy()
    tmp=np.argwhere(np.array(action)>0.5)
    items=tmp.reshape(len(tmp))
    if t<args.card:
        itemlist=userclick[user][:args.card]
    else:
        itemlist=userclick[user][(t-args.card):min((t+args.card),len(userclick[user]))]
    cnt2=0
    for i in items:
        cnt2+=( itemlist.count(i))
    return cnt2/(args.card)

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')

class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=20
        self.card=5
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
args5=para()
thre=0.4
legalList={}
for i in range(20):
    legalList[i]=[]
for i in range(20):
    for j in range(20):
        if i!=j and np.linalg.norm(fixedItemSet[i]-fixedItemSet[j])<thre:
            legalList[i].append(j)
def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(sum(x)-args.card)/args.card/2

In [2]:


def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()

    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))

    MODEL.update()


    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)


    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()

    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()

    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    MODEL.update()

    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()

    
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_mixed(Q,a,MODEL):
    import gurobipy
    import time
    s=time.time()

    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))

    MODEL.update()

    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables))+(np.array(variables).dot(a))*(np.array(variables).dot(a)), sense=gurobipy.GRB.MAXIMIZE)

    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    MODEL.optimize()
    return np.array(MODEL.x)

In [3]:
#g2a
criterion = nn.MSELoss()
class g2apara:
    def __init__(self):
        self.rnn_hidden_dim=8
        self.n_agents=args.solDim
        self.attention_dim=8
        self.cuda=False
        self.n_actions=2
        self.hard=False
args3=g2apara()
hids=torch.from_numpy(np.random.random([args.solDim,args3.rnn_hidden_dim])).float()

def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

class Actorg2a(nn.Module):
    def __init__(self, input_shape, args):
        super(Actorg2a, self).__init__()

        # Encoding
        self.encoding = nn.Linear(input_shape, args.rnn_hidden_dim) 
        self.h = nn.GRUCell(args.rnn_hidden_dim, args.rnn_hidden_dim) 

        self.hard_bi_GRU = nn.GRU(args.rnn_hidden_dim * 2, args.rnn_hidden_dim, bidirectional=True)
        self.hard_encoding = nn.Linear(args.rnn_hidden_dim * 2, 2)  

        # Soft
        self.q = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.k = nn.Linear(args.rnn_hidden_dim, args.attention_dim, bias=False)
        self.v = nn.Linear(args.rnn_hidden_dim, args.attention_dim)

        self.decoding = nn.Linear(args.rnn_hidden_dim + args.attention_dim, args.n_actions)
        self.args = args
        self.input_shape = input_shape
        self.softmax=F.softmax

    def forward(self,state, obs=torch.diag(torch.ones(args.solDim)), hidden_state=hids):
        size = obs.shape[0]  # batch_size * n_agents
        obs_encoding = F.relu(self.encoding(obs))
        h_in = hidden_state.reshape(-1, self.args.rnn_hidden_dim)

        h_out = self.h(obs_encoding, h_in)  # (batch_size * n_agents, args.rnn_hidden_dim)

        if self.args.hard:
            h = h_out.reshape(-1, self.args.n_agents, self.args.rnn_hidden_dim) 
            input_hard = []
            for i in range(self.args.n_agents):
                h_i = h[:, i]  # (batch_size, rnn_hidden_dim)
                h_hard_i = []
                for j in range(self.args.n_agents):  
                    if j != i:
                        h_hard_i.append(torch.cat([h_i, h[:, j]], dim=-1))
                
                h_hard_i = torch.stack(h_hard_i, dim=0)
                input_hard.append(h_hard_i)
                
            input_hard = torch.stack(input_hard, dim=-2)
            
            input_hard = input_hard.view(self.args.n_agents - 1, -1, self.args.rnn_hidden_dim * 2)

            h_hard = torch.zeros((2 * 1, size, self.args.rnn_hidden_dim))  
            if self.args.cuda:
                h_hard = h_hard.cuda()
            h_hard, _ = self.hard_bi_GRU(input_hard, h_hard)  # (n_agents - 1,batch_size * n_agents,rnn_hidden_dim * 2)
            h_hard = h_hard.permute(1, 0, 2)  # (batch_size * n_agents, n_agents - 1, rnn_hidden_dim * 2)
            h_hard = h_hard.reshape(-1, self.args.rnn_hidden_dim * 2)  # (batch_size * n_agents * (n_agents - 1), rnn_hidden_dim * 2)

            
            hard_weights = self.hard_encoding(h_hard)
            hard_weights = F.gumbel_softmax(hard_weights, tau=0.01)
            # print(hard_weights)
            hard_weights = hard_weights[:, 1].view(-1, self.args.n_agents, 1, self.args.n_agents - 1)
            hard_weights = hard_weights.permute(1, 0, 2, 3)

        else:
            hard_weights = torch.ones((self.args.n_agents, size // self.args.n_agents, 1, self.args.n_agents - 1))
            if self.args.cuda:
                hard_weights = hard_weights.cuda()

        # Soft Attention
        q = self.q(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        k = self.k(h_out).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        v = F.relu(self.v(h_out)).reshape(-1, self.args.n_agents, self.args.attention_dim)  # (batch_size, n_agents, args.attention_dim)
        x = []
        for i in range(self.args.n_agents):
            q_i = q[:, i].view(-1, 1, self.args.attention_dim)  
            k_i = [k[:, j] for j in range(self.args.n_agents) if j != i] 
            v_i = [v[:, j] for j in range(self.args.n_agents) if j != i] 

            k_i = torch.stack(k_i, dim=0)  # (n_agents - 1, batch_size, args.attention_dim)
            k_i = k_i.permute(1, 2, 0)  
            v_i = torch.stack(v_i, dim=0)
            v_i = v_i.permute(1, 2, 0)

            # (batch_size, 1, attention_dim) * (batch_size, attention_dim，n_agents - 1) = (batch_size, 1，n_agents - 1)
            score = torch.matmul(q_i, k_i)

            
            scaled_score = score / np.sqrt(self.args.attention_dim)

            
            soft_weight = F.softmax(scaled_score, dim=-1)  # (batch_size，1, n_agents - 1)

            
            x_i = (v_i * soft_weight * hard_weights[i]).sum(dim=-1)
            x.append(x_i)

            
        x = torch.stack(x, dim=1).reshape(-1, self.args.attention_dim)  # (batch_size * n_agents, args.attention_dim)
        final_input = torch.cat([h_out, x], dim=-1)
        output = self.softmax(1*self.decoding(final_input))
        return torch.cat(([i[0].view(1) for i in output]),0)

class Criticg2a(nn.Module):
    def __init__(self, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Criticg2a, self).__init__()
        self.fc1 = nn.Linear(nb_actions, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.init_weights(init_w)

    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out


In [4]:

Q= np.load('Q24.npy')
rewards=[]
constraints=[]
import gym

# https://github.com/openai/gym/blob/master/gym/core.py
class Normalizedenv2(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)
import os
import torch
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
FLOAT = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor


def to_numpy(var):
    return var.cpu().data.numpy() if USE_CUDA else var.data.numpy()

def to_tensor(ndarray, volatile=False, requires_grad=False, dtype=FLOAT):
    return Variable(
        torch.from_numpy(ndarray.astype(np.float64)), volatile=volatile, requires_grad=requires_grad
    ).type(dtype)

def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

def get_output_folder(parent_dir, env2_name):
    """Return save folder.
    Assumes folders in the parent_dir have suffix -run{run
    number}. Finds the highest run number and sets the output folder
    to that number + 1. This is just convenient so that if you run the
    same script multiple times tensorboard can plot all of the results
    on the same plots with different names.
    Parameters
    ----------
    parent_dir: str
      Path of the directory containing all experiment runs.
    Returns
    -------
    parent_dir/run_dir
      Path to this run's save directory.
    """
    os.makedirs(parent_dir, exist_ok=True)
    experiment_id = 0
    for folder_name in os.listdir(parent_dir):
        if not os.path.isdir(os.path.join(parent_dir, folder_name)):
            continue
        try:
            folder_name = int(folder_name.split('-run')[-1])
            if folder_name > experiment_id:
                experiment_id = folder_name
        except:
            pass
    experiment_id += 1

    parent_dir = os.path.join(parent_dir, env2_name)
    parent_dir = parent_dir + '-run{}'.format(experiment_id)
    os.makedirs(parent_dir, exist_ok=True)
    return parent_dir
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
class Evaluator(object):

    def __init__(self, num_episodes, interval, save_path='', max_episode_length=None):
        self.num_episodes = num_episodes
        self.max_episode_length = max_episode_length
        self.interval = interval
        self.save_path = save_path
        self.results = np.array([]).reshape(num_episodes,0)

    def __call__(self, env2, policy,m, debug=False, visualize=False, save=True):

        self.is_training = False
        observation = None
        result = []

        for episode in range(self.num_episodes):

            # reset at the start of episode
            observation = env2.reset()
            episode_steps = 0
            episode_reward = 0.

            assert observation is not None

            # start episode
            done = False
            while not done:
                # basic operation, action ,reward, blablabla ...
                action = policy([observation])
                m.eval()
                observation, reward, done, info = env2.step(action,m)
                if self.max_episode_length and episode_steps >= self.max_episode_length -1:
                    done = True

                # update
                episode_reward += reward
                episode_steps += 1

            result.append(episode_reward)

        result = np.array(result).reshape(-1,1)
        self.results = np.hstack([self.results, result])

        if save:
            self.save_results('{}/validate_reward'.format(self.save_path))
        return np.mean(result)

    def save_results(self, fn):

        y = np.mean(self.results, axis=0)
        error=np.std(self.results, axis=0)

        x = range(0,self.results.shape[1]*self.interval,self.interval)
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        plt.xlabel('Timestep')
        plt.ylabel('Average Reward')
        ax.errorbar(x, y, yerr=error, fmt='-o')
        plt.savefig(fn+'.png')
        savemat(fn+'.mat', {'reward':self.results})
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

#from ipdb import set_trace as debug

def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, nb_actions)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.init_weights(init_w)
        self.sigmoid=nn.Sigmoid()
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out=self.sigmoid(out)
        return out

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=400, hidden2=300, init_w=3e-3):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1 +nb_actions, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()
        self.init_weights(init_w)

    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)

    def forward(self, xs):
        x, a = xs
        out = self.fc1(x)
        out = self.relu(out)
        # debug()
        if len(a.shape)==1:
            a=a.view([1,a.shape[0]])
        print(out.shape,a.shape)
        out = self.fc2(torch.cat([out,a],1))
        out = self.relu(out)
        out = self.fc3(out)
        return out

from collections import deque, namedtuple
import warnings
import random

import numpy as np
from sklearn.datasets import make_classification
from sklearn.cluster import Birch
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/memory.py

# This is to be understood as a transition: Given `state0`, performing `action`
# yields `reward` and results in `state1`, which might be `terminal`.
Experience = namedtuple('Experience', 'state0, action, reward, state1, terminal1')


def sample_batch_indexes(low, high, size):
    if high - low >= size:
        # We have enough data. Draw without replacement, that is each index is unique in the
        # batch. We cannot use `np.random.choice` here because it is horribly inefficient as
        # the memory grows. See https://github.com/numpy/numpy/issues/2764 for a discussion.
        # `random.sample` does the same thing (drawing without replacement) and is way faster.
        try:
            r = xrange(low, high)
        except NameError:
            r = range(low, high)
        batch_idxs = random.sample(r, size)
    else:
        # Not enough data. Help ourselves with sampling from the range, but the same index
        # can occur multiple times. This is not good and should be avoided by picking a
        # large enough warm-up phase.
        warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
        batch_idxs = np.random.random_integers(low, high - 1, size=size)
    assert len(batch_idxs) == size
    return batch_idxs


class RingBuffer(object):
    def __init__(self, maxlen):
        self.maxlen = maxlen
        self.start = 0
        self.length = 0
        self.data = [None for _ in range(maxlen)]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise KeyError()
        return self.data[(self.start + idx) % self.maxlen]

    def append(self, v):
        if self.length < self.maxlen:
            # We have space, simply increase the length.
            self.length += 1
        elif self.length == self.maxlen:
            # No space, "remove" the first item.
            self.start = (self.start + 1) % self.maxlen
        else:
            # This should never happen.
            raise RuntimeError()
        self.data[(self.start + self.length - 1) % self.maxlen] = v


def zeroed_observation(observation):
    if hasattr(observation, 'shape'):
        return np.zeros(observation.shape)
    elif hasattr(observation, '__iter__'):
        out = []
        for x in observation:
            out.append(zeroed_observation(x))
        return out
    else:
        return 0.


class Memory(object):
    def __init__(self, window_length, ignore_episode_boundaries=False):
        self.window_length = window_length
        self.ignore_episode_boundaries = ignore_episode_boundaries

        self.recent_observations = deque(maxlen=window_length)
        self.recent_terminals = deque(maxlen=window_length)

    def sample(self, batch_size, batch_idxs=None):
        raise NotImplementedError()

    def append(self, observation, action, reward, terminal, training=True):
        self.recent_observations.append(observation)
        self.recent_terminals.append(terminal)

    def get_recent_state(self, current_observation):
        # This code is slightly complicated by the fact that subsequent observations might be
        # from different episodes. We ensure that an experience never spans multiple episodes.
        # This is probably not that important in practice but it seems cleaner.
        state = [current_observation]
        idx = len(self.recent_observations) - 1
        for offset in range(0, self.window_length - 1):
            current_idx = idx - offset
            current_terminal = self.recent_terminals[current_idx - 1] if current_idx - 1 >= 0 else False
            if current_idx < 0 or (not self.ignore_episode_boundaries and current_terminal):
                # The previously handled observation was terminal, don't add the current one.
                # Otherwise we would leak into a different episode.
                break
            state.insert(0, self.recent_observations[current_idx])
        while len(state) < self.window_length:
            state.insert(0, zeroed_observation(state[0]))
        return state

    def get_config(self):
        config = {
            'window_length': self.window_length,
            'ignore_episode_boundaries': self.ignore_episode_boundaries,
        }
        return config
def samplewithclustering(acs,size):
    model = Birch(threshold=0.01, n_clusters=20)
    model.fit(acs)      
    yhat = model.predict(acs)
    ydict={}
    for i in range(20):
        ydict[i]=[]
        ydict[i]=ydict[i]+list(np.array(np.argwhere(yhat==i)).reshape(len(np.argwhere(yhat==i))))
    ids=[]
    for i in range(20):
        np.random.shuffle(ydict[i])
        ids+=ydict[i][:int(size*len(ydict[i])/len(yhat))]
    return ids
class SequentialMemory(Memory):
    def __init__(self, limit, **kwargs):
        super(SequentialMemory, self).__init__(**kwargs)

        self.limit = limit

        # Do not use deque to implement the memory. This data structure may seem convenient but
        # it is way too slow on random access. Instead, we use our own ring buffer implementation.
        self.actions = RingBuffer(limit)
        self.rewards = RingBuffer(limit)
        self.terminals = RingBuffer(limit)
        self.observations = RingBuffer(limit)

    def sample(self, batch_size, batch_idxs=None):
        if batch_idxs is None:
            # Draw random indexes such that we have at least a single entry before each
            # index.
            sample1s=samplewithclustering([list(i) for i in self.actions.data[:(self.actions.start + self.actions.length - 1) % self.actions.maxlen]],batch_size//2)
            batch_idxs = sample1s+sample_batch_indexes(0, self.nb_entries - 1, size=batch_size-len(sample1s))
        batch_idxs = np.array(batch_idxs) + 1
        assert np.min(batch_idxs) >= 1
        assert np.max(batch_idxs) < self.nb_entries
        assert len(batch_idxs) == batch_size

        # Create experiences
        experiences = []
        for idx in batch_idxs:
            terminal0 = self.terminals[idx - 2] if idx >= 2 else False
            while terminal0:
                # Skip this transition because the env2ironment was reset here. Select a new, random
                # transition and use this instead. This may cause the batch to contain the same
                # transition twice.
                idx = sample_batch_indexes(1, self.nb_entries, size=1)[0]
                terminal0 = self.terminals[idx - 2] if idx >= 2 else False
            assert 1 <= idx < self.nb_entries

            # This code is slightly complicated by the fact that subsequent observations might be
            # from different episodes. We ensure that an experience never spans multiple episodes.
            # This is probably not that important in practice but it seems cleaner.
            state0 = [self.observations[idx - 1]]
            for offset in range(0, self.window_length - 1):
                current_idx = idx - 2 - offset
                current_terminal = self.terminals[current_idx - 1] if current_idx - 1 > 0 else False
                if current_idx < 0 or (not self.ignore_episode_boundaries and current_terminal):
                    # The previously handled observation was terminal, don't add the current one.
                    # Otherwise we would leak into a different episode.
                    break
                state0.insert(0, self.observations[current_idx])
            while len(state0) < self.window_length:
                state0.insert(0, zeroed_observation(state0[0]))
            action = self.actions[idx - 1]
            reward = self.rewards[idx - 1]
            terminal1 = self.terminals[idx - 1]

            # Okay, now we need to create the follow-up state. This is state0 shifted on timestep
            # to the right. Again, we need to be careful to not include an observation from the next
            # episode if the last state is terminal.
            state1 = [np.copy(x) for x in state0[1:]]
            state1.append(self.observations[idx])

            assert len(state0) == self.window_length
            assert len(state1) == len(state0)
            experiences.append(Experience(state0=state0, action=action, reward=reward,
                                          state1=state1, terminal1=terminal1))
        assert len(experiences) == batch_size
        return experiences

    def sample_and_split(self, batch_size, batch_idxs=None):
        experiences = self.sample(batch_size, batch_idxs)

        state0_batch = []
        reward_batch = []
        action_batch = []
        terminal1_batch = []
        state1_batch = []
        for e in experiences:
            state0_batch.append(e.state0)
            state1_batch.append(e.state1)
            reward_batch.append(e.reward)
            action_batch.append(e.action)
            terminal1_batch.append(0. if e.terminal1 else 1.)

        # Prepare and validate parameters.
        state0_batch = np.array(state0_batch).reshape(batch_size,-1)
        state1_batch = np.array(state1_batch).reshape(batch_size,-1)
        terminal1_batch = np.array(terminal1_batch).reshape(batch_size,-1)
        reward_batch = np.array(reward_batch).reshape(batch_size,-1)
        action_batch = np.array(action_batch).reshape(batch_size,-1)

        return state0_batch, action_batch, reward_batch, state1_batch, terminal1_batch


    def append(self, observation, action, reward, terminal, training=True):
        super(SequentialMemory, self).append(observation, action, reward, terminal, training=training)

        # This needs to be understood as follows: in `observation`, take `action`, obtain `reward`
        # and weather the next state is `terminal` or not.
        if training:
            
            self.observations.append(observation)
            
            self.actions.append(action)
            self.rewards.append(reward)
            self.terminals.append(terminal)

    @property
    def nb_entries(self):
        return len(self.observations)

    def get_config(self):
        config = super(SequentialMemory, self).get_config()
        config['limit'] = self.limit
        return config


class EpisodeParameterMemory(Memory):
    def __init__(self, limit, **kwargs):
        super(EpisodeParameterMemory, self).__init__(**kwargs)
        self.limit = limit

        self.params = RingBuffer(limit)
        self.intermediate_rewards = []
        self.total_rewards = RingBuffer(limit)

    def sample(self, batch_size, batch_idxs=None):
        if batch_idxs is None:
            batch_idxs = sample_batch_indexes(0, self.nb_entries, size=batch_size)
        assert len(batch_idxs) == batch_size

        batch_params = []
        batch_total_rewards = []
        for idx in batch_idxs:
            batch_params.append(self.params[idx])
            batch_total_rewards.append(self.total_rewards[idx])
        return batch_params, batch_total_rewards

    def append(self, observation, action, reward, terminal, training=True):
        super(EpisodeParameterMemory, self).append(observation, action, reward, terminal, training=training)
        if training:
            self.intermediate_rewards.append(reward)

    def finalize_episode(self, params):
        total_reward = sum(self.intermediate_rewards)
        self.total_rewards.append(total_reward)
        self.params.append(params)
        self.intermediate_rewards = []

    @property
    def nb_entries(self):
        return len(self.total_rewards)

    def get_config(self):
        config = super(SequentialMemory, self).get_config()
        config['limit'] = self.limit
        return config
import numpy as np 

# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)
# from ipdb import set_trace as debug

criterion = nn.MSELoss()

class DDPG(object):
    def __init__(self, nb_states, nb_actions, args2):

        if args2.seed > 0:
            self.seed(args2.seed)

        self.nb_states = nb_states
        self.nb_actions= nb_actions

        # Create Actor and Critic Network
        net_cfg = {
            'hidden1':args2.hidden1, 
            'hidden2':args2.hidden2, 
            'init_w':args2.init_w
        }
        self.actor = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_target = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_optim  = Adam(self.actor.parameters(), lr=args2.prate)

        self.critic = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_target = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_optim  = Adam(self.critic.parameters(), lr=args2.rate)

        hard_update(self.actor_target, self.actor) # Make sure target is with the same weight
        hard_update(self.critic_target, self.critic)

        #Create replay buffer
        self.memory = SequentialMemory(limit=args2.rmsize, window_length=args2.window_length)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=args2.ou_theta, mu=args2.ou_mu, sigma=args2.ou_sigma)

        # Hyper-parameters
        self.batch_size = args2.bsize
        self.tau = args2.tau
        self.discount = args2.discount
        self.depsilon = 1.0 / args2.epsilon

        # 
        self.epsilon = 1.0
        self.s_t = None # Most recent state
        self.a_t = None # Most recent action
        self.is_training = True

        # 
        if USE_CUDA: self.cuda()

    def update_policy(self):
        # Sample batch
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.sample_and_split(self.batch_size)

        # Prepare for the target q batch
        next_q_values = self.critic_target([
            to_tensor(next_state_batch, volatile=True),
            self.actor_target(to_tensor(next_state_batch, volatile=True)),
        ])
        next_q_values.volatile=False

        target_q_batch = to_tensor(reward_batch) + \
            self.discount*to_tensor(terminal_batch.astype(np.float))*next_q_values

        # Critic update
        self.critic.zero_grad()

        q_batch = self.critic([ to_tensor(state_batch), to_tensor(action_batch) ])

        value_loss = criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        # Actor update
        self.actor.zero_grad()

        policy_loss = -self.critic([
            to_tensor(state_batch),
            self.actor(to_tensor(state_batch))
        ])

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        # Target update
        soft_update(self.actor_target, self.actor, self.tau)
        soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def cuda(self):
        self.actor.cuda()
        self.actor_target.cuda()
        self.critic.cuda()
        self.critic_target.cuda()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            #print('----',self.a_t)
            self.memory.append(self.s_t, self.a_t, r_t, done)
            self.s_t = s_t1

    def random_action(self):
        action = np.random.uniform(0.,1.,self.nb_actions)
        self.a_t = action
        return action

    def select_action(self, s_t, decay_epsilon=True):
        #print('self.actor(to_tensor(np.array([s_t]))):',self.actor(to_tensor(np.array([s_t]))))
        action = to_numpy(
            self.actor(to_tensor(np.array([s_t])))
        ).squeeze(0)
        #action += self.is_training*max(self.epsilon, 0)*self.random_process.sample()
        action = np.clip(action, 0., 1.)

        if decay_epsilon:
            self.epsilon -= self.depsilon

        self.a_t = action
        
        return action

    def reset(self, obs):
        self.s_t = obs
        self.random_process.reset_states()

    def load_weights(self, output):
        if output is None: return

        self.actor.load_state_dict(
            torch.load('{}/actor.pkl'.format(output))
        )

        self.critic.load_state_dict(
            torch.load('{}/critic.pkl'.format(output))
        )


    def save_model(self,output):
        torch.save(
            self.actor.state_dict(),
            '{}/actor.pkl'.format(output)
        )
        torch.save(
            self.critic.state_dict(),
            '{}/critic.pkl'.format(output)
        )

    def seed(self,s):
        torch.manual_seed(s)
        if USE_CUDA:
            torch.cuda.manual_seed(s)
import pyflann
from gym.spaces import Box
import numpy as np
import itertools

class Space:

    def __init__(self, low, high, points):

        self._low = np.array(low)
        self._high = np.array(high)
        self._range = self._high - self._low
        self._dimensions = len(low)
        self.__space = init_uniform_space([0] * self._dimensions,
                                          [1] * self._dimensions,
                                          points)
        self._flann = pyflann.FLANN()
        self.rebuild_flann()

    def rebuild_flann(self):
        self._index = self._flann.build_index(self.__space, algorithm='kdtree')
        
    def search_point(self, point, k):
        p_in = self.import_point(point).reshape(1, -1).astype('float64')
        search_res, _ = self._flann.nn_index(p_in, k)
        knns = self.__space[search_res]
        p_out = []
        for p in knns:
            p_out.append(self.export_point(p))

        if k == 1:
            p_out = [p_out]
        return np.array(p_out)

    def import_point(self, point):
        return (point - self._low) / self._range

    def export_point(self, point):
        return self._low + point * self._range

    def get_space(self):
        return self.__space

    def shape(self):
        return self.__space.shape

    def get_number_of_actions(self):
        return self.shape()[0]

    def plot_space(self, additional_points=None):

        dims = self._dimensions

        if dims > 3:
            print(
                'Cannot plot a {}-dimensional space. Max 3 dimensions'.format(dims))
            return

        space = self.get_space()
        if additional_points is not None:
            for i in additional_points:
                space = np.append(space, additional_points, axis=0)

        if dims == 1:
            for x in space:
                plt.plot([x], [0], 'o')

            plt.show()
        elif dims == 2:
            for x, y in space:
                plt.plot([x], [y], 'o')

            plt.show()
        else:
            plot_3d_points(space)


class Discrete_space(Space):
    """
        Discrete action space with n actions (the integers in the range [0, n))
        0, 1, 2, ..., n-2, n-1
    """

    def __init__(self, n):  # n: the number of the discrete actions
        super().__init__([0], [n - 1], n)

    def export_point(self, point):
        return super().export_point(point).astype(int)


def init_uniform_space(low, high, points):
    dims = len(low)
    points_in_each_axis = round(points**(1 / dims))

    axis = []
    for i in range(dims):
        axis.append(list(np.linspace(low[i], high[i], points_in_each_axis)))
    space = []
    for _ in itertools.product(*axis):
        space.append(list(_))
    return np.array(space)
import numpy as np

import torch
import torch.nn as nn
from torch.optim import Adam
criterion = nn.MSELoss()

class WOLPAgent(object):
    def __init__(self, nb_states, nb_actions, args):

        if args2.seed > 0:
            self.seed(args2.seed)

        self.nb_states = nb_states
        self.nb_actions= nb_actions

        # Create Actor and Critic Network
        net_cfg = {
            'hidden1':args2.hidden1, 
            'hidden2':args2.hidden2, 
            'init_w':args2.init_w
        }

        ################################## Our Code Start ################################################
        self.low = args2.low
        self.high = args2.high
        self.action_space = spaces.Box(
            low=0,
            high=1,
            shape=(args5.solDim,)
        )
        self.k_nearest_neighbors = max(1, int(args2.max_actions * args2.k_ratio))
        ################################## Our Code End ################################################        

        self.actor = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_target = Actor(self.nb_states, self.nb_actions, **net_cfg)
        self.actor_optim  = Adam(self.actor.parameters(), lr=args2.prate)
        self.critic = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_target = Critic(self.nb_states, self.nb_actions, **net_cfg)
        self.critic_optim  = Adam(self.critic.parameters(), lr=args2.rate)

        hard_update(self.actor_target, self.actor) # Make sure target is with the same weight
        hard_update(self.critic_target, self.critic)

        #Create replay buffer
        self.memory = SequentialMemory(limit=args2.rmsize, window_length=args2.window_length)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=args2.ou_theta, mu=args2.ou_mu, sigma=args2.ou_sigma)

        # Hyper-parameters
        self.batch_size = args2.bsize
        self.tau = args2.tau
        self.discount = args2.discount
        self.depsilon = 1.0 / args2.epsilon

        # 
        self.epsilon = 1.0
        self.s_t = None # Most recent state
        self.a_t = None # Most recent action
        self.is_training = True

        # 
        if USE_CUDA: self.cuda()

    def get_action_space(self):
        return self.action_space    

    def update_policy(self):
        # Sample batch
        state_batch, action_batch, reward_batch, \
        next_state_batch, terminal_batch = self.memory.sample_and_split(self.batch_size)

        # Prepare for the target q batch
        next_q_values = self.critic_target([
            to_tensor(next_state_batch, volatile=True),
            self.actor_target(to_tensor(next_state_batch, volatile=True)),
        ])
        next_q_values.volatile=False

        target_q_batch = to_tensor(reward_batch) + \
            self.discount*to_tensor(terminal_batch.astype(np.float))*next_q_values

        # Critic update
        self.critic.zero_grad()
        q_batch = self.critic([ to_tensor(state_batch), to_tensor(action_batch) ])

        value_loss = criterion(q_batch, target_q_batch)
        value_loss.backward()
        self.critic_optim.step()

        # Actor update
        self.actor.zero_grad()

        policy_loss = -self.critic([
            to_tensor(state_batch),
            self.actor(to_tensor(state_batch))
        ])

        policy_loss = policy_loss.mean()
        policy_loss.backward()
        self.actor_optim.step()

        # Target update
        soft_update(self.actor_target, self.actor, self.tau)
        soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def cuda(self):
        self.actor.cuda()
        self.actor_target.cuda()
        self.critic.cuda()
        self.critic_target.cuda()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            self.memory.append(self.s_t, self.a_t, r_t, done)
            self.s_t = s_t1

    def random_action(self):
        action = np.random.uniform(0.,1.,self.nb_actions)
        self.a_t = action
        return action

    def select_action(self, s_t, decay_epsilon=True):
        proto_action = self.ddpg_select_action(s_t, decay_epsilon=decay_epsilon)
        actions = proto_action
        
        states = np.array([[0.1]*args5.solDim])

        a = [to_tensor(states), to_tensor(actions)]
        # print("states: {}, actions: {}".format(a[0].size(), a[1].size()))
        actions_evaluation = self.critic([to_tensor(states), to_tensor(actions)])
        # print("actions_evaluation: {}, actions_evaluation.size(): {}".format(actions_evaluation, actions_evaluation.size()))
        actions_evaluation_np = actions_evaluation.cpu().data.numpy()
        max_index = np.argmax(actions_evaluation_np)

        self.a_t = actions[max_index]
        return self.a_t

    def ddpg_select_action(self, s_t, decay_epsilon=True):
        action = to_numpy(
            self.actor(to_tensor(np.array([s_t])))
        ).squeeze(0)            
        #action += self.is_training*max(self.epsilon, 0)*self.random_process.sample()
        action = np.clip(action, 0., 1.)
        if decay_epsilon:
            self.epsilon -= self.depsilon
        return action
    def select_action2(self, s_t, decay_epsilon=True):
        proto_action = self.ddpg_select_action2(s_t, decay_epsilon=decay_epsilon)
        actions = proto_action
        
        states = np.array([[0.1]*args5.solDim])

        # print("states: {}, actions: {}".format(a[0].size(), a[1].size()))
        actions_evaluation = self.critic([to_tensor(states), to_tensor(actions.detach().numpy())])
        # print("actions_evaluation: {}, actions_evaluation.size(): {}".format(actions_evaluation, actions_evaluation.size()))
        actions_evaluation_np = actions_evaluation.cpu().data.numpy()
        max_index = np.argmax(actions_evaluation_np)

        self.a_t = actions[max_index]
        return self.a_t
    def ddpg_select_action2(self, s_t, decay_epsilon=True):
        action = self.actor(to_tensor(np.array([s_t])))          
        #action += self.is_training*max(self.epsilon, 0)*self.random_process.sample()
        action = torch.clamp(action, 0., 1.)
        if decay_epsilon:
            self.epsilon -= self.depsilon
        return action    
    def reset(self, obs):
        self.s_t = obs
        self.random_process.reset_states()

    def load_weights(self, output):
        if output is None: return

        self.actor.load_state_dict(
            torch.load('{}/actor.pkl'.format(output))
        )

        self.critic.load_state_dict(
            torch.load('{}/critic.pkl'.format(output))
        )


    def save_model(self,output):
        torch.save(
            self.actor.state_dict(),
            '{}/actor.pkl'.format(output)
        )
        torch.save(
            self.critic.state_dict(),
            '{}/critic.pkl'.format(output)
        )

    def seed(self,s):
        torch.manual_seed(s)
        if USE_CUDA:
            torch.cuda.manual_seed(s)
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

   
class Arguments(object):
    def __init__(self):
        self.mode = 'train'
        self.env2 = "InvertedPendulum-v2"
        self.hidden1 = 400
        self.hidden2 = 300
        self.rate = 0.001
        self.prate = 0.0001
        self.warmup = 100
        self.discount = 0.99
        self.bsize = 64
        self.rmsize = 6000000
        self.window_length = 1
        self.tau = 0.001
        self.ou_theta = 0.15
        self.ou_sigma = 0.2
        self.ou_mu = 0.0
        self.validate_episodes = 20
        self.max_episode_length = 500
        self.validate_steps = 5000
        self.output = 'output'
        self.debug='debug'
        self.init_w = 0.03
        self.train_iter=20000
        self.epsilon=50000
        self.seed=-1
        self.max_actions=1e6
        self.resume='default'
        self.k_ratio = 1e-6

args2 = Arguments()
args2.output = get_output_folder(args2.output, args2.env2)
if args2.resume == 'default':
    args2.resume = 'output/{}-run0'.format(args2.env2)
class myenv2(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds

        self.action_space =  spaces.Box(
            low=0,
            high=1,
            shape=(args5.solDim,)
        )
        self.observation_space =  spaces.Box(
            low=0,
            high=1,
            shape=(args5.solDim,)
        )

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

        self.steps = torch.tensor(np.array([0.1]*args5.solDim))
        self._max_episode_steps = 500
        self.iteration=1
        self.a = 10*(np.load('rateListUsersYahoo.npy')[0]-np.load('rateListUsersYahoo.npy')[0].min())
        self.a /= np.linalg.norm(self.a, ord=2)
        self.constraints=[]
        self.constraint=0
        self.history=np.ones(args5.solDim)
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action,m):
        self.iteration+=1 
        if self.iteration==60:
            self.constraint=10#10*sum(rewards)/len(rewards)
            self.std=np.array(rewards).std()
        if self.iteration>1 and self.iteration%60==0:
            self.constraint=min(100*sum(rewards)/len(rewards),max(0,self.constraint + 0.005*(-0.0+sum(self.constraints)/len(self.constraints))))
        done=1
        if len(action)==1:
            action=action[0]
        action=[action[i]+0.01*np.sqrt(np.log(self.iteration)/self.history[i]) for i in range(args.solDim)]
        action=np.array([1 if i in np.argsort(action)[-args.card:] else 0  for i in range(args.solDim)])
        
        self.constraints.append(checkFea(action))
        #print(action,checkFea(action))
  #      if sum(action)<10:
   #         reward=100*m.forward(torch.FloatTensor(action).to(self.device)).detach().squeeze()-100*(sum(action)-10)**2+100000#+ 100*(1/(1+np.exp(-self.iteration))-0.5)*torch.log(torch.mean(wmc)).float()#0*(1/(1+np.exp(-self.iteration))-0.5)*torch.log(torch.mean(wmc)).float()+sum(action)#100*m.forward(torch.FloatTensor(action).to(self.device)).detach().squeeze()
    #    else:
        #m.forward(torch.FloatTensor(action).to(self.device)).detach().squeeze()
        m.eval()
        reward= m.forward(torch.FloatTensor(action)).detach().squeeze()-self.constraint*checkFea(action)##0*(1/(1+np.exp(-self.iteration))-0.5)*torch.log(torch.mean(wmc)).float()+sum(action)#100*m.forward(torch.FloatTensor(action).to(self.device)).detach().squeeze()
        rewards.append(m.forward(torch.FloatTensor(action)).detach().squeeze())
        constraints.append(checkFea(action))
        self.history+=np.array(action)
        #print(self.iteration,1*(self.a.dot(action))**1/(np.sqrt(10))**1,checkFea(action),action)
        #print(m.forward(torch.FloatTensor(action).to(self.device)).detach().squeeze(),checkFea(action),torch.log(torch.mean(wmc)))
        return action, reward, done, {}

    def reset(self):
        return  np.array([0.1]*args5.solDim).reshape([1,args5.solDim])

    def render(self, mode='human'):
        pass 

In [5]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

def inv_sherman_morrison(u, A_inv):
    """Inverse of a matrix with rank 1 update.
    """
    Au = np.dot(A_inv, u)
    A_inv -= np.outer(Au, Au)/(1+np.dot(u.T, Au))
    return A_inv

class Model(nn.Module):
    """Template for fully connected neural network for scalar approximation.
    """
    def __init__(self, 
                 input_size=1, 
                 hidden_size=2,
                 n_layers=4,
                 activation='ReLU',
                 p=0.0,
                ):
        super(Model, self).__init__()
        
        self.n_layers = n_layers
        
        if self.n_layers == 1:
            self.layers = [nn.Linear(input_size, 1)]
        else:
            size  = [input_size] + [hidden_size,] * (self.n_layers-1) + [1]
            self.layers = [nn.Linear(size[i], size[i+1]) for i in range(self.n_layers)]
        self.layers = nn.ModuleList(self.layers)
        
        # dropout layer
        self.dropout = nn.Dropout(p=p)
        
        # activation function
        if activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'LeakyReLU':
            self.activation = nn.LeakyReLU(negative_slope=0.1)
        else:
            raise Exception('{} not an available activation'.format(activation))
            
    def forward(self, x):
        for i in range(self.n_layers-1):
            x = self.activation(self.layers[i](x))
        x = self.layers[-1](x)
        return x


In [6]:
import numpy as np
import abc
from tqdm import tqdm
class UCB(abc.ABC):
    """Base class for UBC methods.
    """
    def __init__(self,
                 bandit,
                 reg_factor=1.0,
                 confidence_scaling_factor=-1.0,
                 delta=0.1,
                 train_every=1,
                 throttle=int(1e2),
                ):
        # bandit object, contains features and generated rewards
        self.bandit = bandit
        # L2 regularization strength
        self.reg_factor = reg_factor
        # Confidence bound with probability 1-delta
        self.delta = delta
        # multiplier for the confidence bound (default is bandit reward noise std dev)
        if confidence_scaling_factor == -1.0:
            confidence_scaling_factor = bandit.noise_std
        self.confidence_scaling_factor = confidence_scaling_factor
        
        # train approximator only every few rounds
        self.train_every = train_every
        self.best_recommended_reward =-200
        self.best_recommended_action =np.array([1]*args.card+[0]*(self.bandit.n_features-args.card))
        self.rewards_list=[]
        self.best_sample_rewards_list=[]
        self.feasi=[]
        # throttle tqdm updates
        self.throttle = throttle
        self.solverSol=None
        self.reset()
        
    def reset_upper_confidence_bounds(self):
        """Initialize upper confidence bounds and related quantities.
        """
        self.exploration_bonus = np.empty((self.bandit.n_arms))
        self.mu_hat = np.empty((self.bandit.n_arms)) 
        self.meta_mu_hat = np.empty(( self.bandit.n_features)) 
        self.cross_mu_hat = np.empty(( self.bandit.n_features**2)) 
        self.upper_confidence_bounds = np.ones((self.bandit.n_arms))
        self.meta_values= np.ones((self.bandit.n_features))
        self.cross_values= np.ones((self.bandit.n_features**2))
        self.knn_metabest=-100
        self.best_rewards_oracle=-100

    def reset_actions(self):
        """Initialize cache of actions.
        """
        self.actions = np.empty(self.bandit.T).astype('int')
    
    def reset_A_inv(self):
        """Initialize n_arms square matrices representing the inverses
        of exploration bonus matrices.
        """
        self.A_inv = np.array(
            [
                np.eye(self.approximator_dim)/self.reg_factor for _ in self.bandit.arms
            ]
        )
    
    def reset_grad_approx(self):
        """Initialize the gradient of the approximator w.r.t its parameters.
        """
        self.grad_approx = np.zeros((self.bandit.n_arms, self.approximator_dim))

    def sample_action(self):
        """Return the action to play based on current estimates
        """
        if self.iteration>=max(self.bandit.n_features*2,100):
            return int(torch.argmax(self.upper_confidence_bounds).item())
        else:
            return np.random.choice(self.bandit.n_arms)
    
    @abc.abstractmethod
    def reset(self):
        """Initialize variables of interest.
        To be defined in children classes.
        """
        pass

    @property
    @abc.abstractmethod
    def approximator_dim(self):
        """Number of parameters used in the approximator.
        """
        pass
    
    @property
    @abc.abstractmethod
    def confidence_multiplier(self):
        """Multiplier for the confidence exploration bonus.
        To be defined in children classes.
        """
        pass
    
    @abc.abstractmethod
    def update_confidence_bounds(self):
        """Update the confidence bounds for all arms at time t.
        To be defined in children classes.
        """
        pass

    @abc.abstractmethod
    def update_output_gradient(self):
        """Compute output gradient of the approximator w.r.t its parameters.
        """
        pass
    
    @abc.abstractmethod
    def train(self):
        """Update approximator.
        To be defined in children classes.
        """
        pass
    
    @abc.abstractmethod
    def predict(self):
        """Predict rewards based on an approximator.
        To be defined in children classes.
        """
        pass

    def update_confidence_bounds(self):
        """Update confidence bounds and related quantities for all arms.
        """
        if self.iteration>max(self.bandit.n_features*2,100):#self.bandit.n_features:
            self.bandit.features[self.iteration][2]=self.best_recommended_action
            self.bandit.features[self.iteration][0]=self.solverSol
            self.bandit.features[self.iteration][1]=self.solverSol_quad
            for j in range(3,self.bandit.n_arms//3):
                tmp=[np.random.beta(min(1-0.1,max(0.1,mm)),1-min(1-0.1,max(0.1,mm))) for mm in self.solverSol]#[self.meta_values[i]+2*np.sqrt(np.log(self.iteration+1)/self.cnt[i]/self.iteration) for i in range(self.bandit.n_features)]
                tmp2=sorted(tmp)[-args.card:] 
                self.bandit.features[self.iteration][j]= np.divide(np.array([1 if tmp[i] in tmp2 else 0 for i in range(self.bandit.n_features)]),np.sqrt(args.card))
            for j in range(n_arms//3,self.bandit.n_arms//2):
                tmp=[np.random.beta(min(1-0.1,max(0.1,mm)),1-min(1-0.1,max(0.1,mm))) for mm in self.solverSol_quad]#[self.meta_values[i]+2*np.sqrt(np.log(self.iteration+1)/self.cnt[i]/self.iteration) for i in range(self.bandit.n_features)]
                tmp2=sorted(tmp)[-args.card:] 
                self.bandit.features[self.iteration][j]= np.divide(np.array([1 if tmp[i] in tmp2 else 0 for i in range(self.bandit.n_features)]),np.sqrt(args.card))
            for j in range(n_arms//2,2*self.bandit.n_arms//3):
                tmp=self.bandit.features[self.iteration][j]+np.random.random()*(self.best_recommended_action-self.bandit.features[self.iteration][j])
                self.model.eval()
                if self.model.forward(torch.FloatTensor(self.bandit.features[self.iteration][j]).to(self.device)).detach().squeeze()<self.model.forward(torch.FloatTensor(tmp).to(self.device)).detach().squeeze():
                    tmp2=sorted(tmp)[-args.card:] 
                    self.bandit.features[self.iteration][j]= np.divide(np.array([1 if tmp[i] in tmp2 else 0 for i in range(self.bandit.n_features)]),np.sqrt(args.card))
            if len(self.elite)>=5:
                
                for kk in range(min(len(self.elite),self.bandit.n_arms//2)):
                    
                    self.bandit.features[self.iteration][-kk]= self.elite[kk]
            if np.random.random()>0.4:
                a=np.random.choice(self.bandit.n_arms)
                b=np.random.choice(self.bandit.n_arms)
                if a!=b:
                    self.model.eval()
                    if  self.model.forward(torch.FloatTensor(self.bandit.features[self.iteration][a]).to(self.device)).detach().squeeze()>self.model.forward(torch.FloatTensor(self.bandit.features[self.iteration][b]).to(self.device)).detach().squeeze():
                        tmp=self.bandit.features[self.iteration][b]+np.random.random()*(a-b)
                        tmp2=sorted(tmp)[-args.card:] 
                        self.bandit.features[self.iteration][b]= np.divide(np.array([1 if tmp[i] in tmp2 else 0 for i in range(self.bandit.n_features)]),np.sqrt(args.card))



        self.update_output_gradient()
        
        # UCB exploration bonus
        self.exploration_bonus = np.array(
            [
                 -10.*checkFea(self.bandit.features[self.iteration][a]*np.sqrt(args.card))+0.01* np.sqrt(np.dot(self.grad_approx[a], np.dot(self.A_inv[a], self.grad_approx[a].T))) for a in self.bandit.arms
            ]
        )
        self.model.eval()
        self.mu_hat= self.model.forward(
            torch.FloatTensor(self.bandit.features[self.iteration]).to(self.device)
        ).detach().squeeze()
        #print(self.bandit.features[self.iteration][0][:10],self.bandit.features[self.iteration][-1][:10])
        self.meta_mu_hat = self.model.forward(
            torch.FloatTensor(self.bandit.meta_features[self.iteration]).to(self.device)
        ).detach().squeeze()
        if self.iteration>=max(self.bandit.n_features*2,200):
            self.cross_mu_hat= self.model.forward(
                torch.FloatTensor(self.bandit.cross_features).to(self.device)
            ).detach().squeeze()
        self.bandit.rewards[self.iteration]=np.array([self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.bandit.features[ self.iteration,k]) + self.bandit.noise_std*np.random.randn() for k in range(self.bandit.n_arms)])
        # estimated combined bound for reward
        self.meta_values=self.meta_mu_hat# + self.meta_exploration_bonus[self.iteration] 
        if self.iteration>=max(self.bandit.n_features*2,100):
            self.cross_values=self.cross_mu_hat
            for i in range(self.bandit.n_features):
                for j in range(self.bandit.n_features):
                    if i==j:
                        self.cross_values[i*self.bandit.n_features+j]=self.meta_mu_hat[i]
                    else:
                        self.cross_values[i*self.bandit.n_features+j]=(self.cross_mu_hat[i*self.bandit.n_features+j]-self.meta_mu_hat[i]-self.meta_mu_hat[j])/2
            self.cross_values=self.cross_values.reshape(self.bandit.n_features,self.bandit.n_features)
        with gurobipy.Env(empty=True) as env:
            env.setParam('OutputFlag', 0)
            env.setParam('IterationLimit',600)
            env.start()
            with gurobipy.Model(env=env) as m:
                self.solverSol=np.divide(solver(self.meta_values,m),np.sqrt(args.card))
        if self.iteration>=max(self.bandit.n_features*2,100):
            with gurobipy.Env(empty=True) as env:
                env.setParam('OutputFlag', 0)
                env.setParam('IterationLimit',600)
                env.start()
                if sum(sum(np.isnan(model.cross_values)))==0 and sum(sum(np.isinf(model.cross_values)))==0:
                    with gurobipy.Model(env=env) as m:
                        self.solverSol_quad=np.divide(solver_quad(self.cross_values,m),np.sqrt(args.card))
                        #print('solverSol_quad',self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.solverSol_quad))
                else:
                    self.solverSol_quad=self.bandit.features[self.iteration][0]
        tmp=sorted(self.meta_values )[-args.card:]  
        self.meta_values = np.array([1 if self.meta_values[i] in tmp else 0 for i in range(self.bandit.n_features)])
        if len(np.argwhere(self.meta_values==1)):
            metabest=np.argwhere(self.meta_values==1)[:args.card].reshape(args.card)
        else:
            metabest=list(range(self.bandit.n_features))
            np.random.shuffle(metabest)
            metabest=np.array(metabest[:args.card])
        knn_metabest={}
        import copy
        for i in range(1):
            tmp=metabest.copy()
            tmp[np.random.choice(range(args.card))]=np.random.choice(range(self.bandit.n_features))
            tmp=np.divide(np.array([1 if i in tmp else 0 for i in range(self.bandit.n_features)]),np.sqrt(args.card))
            self.model.eval()
            tmpvalue=self.model.forward(
                    torch.FloatTensor(tmp).to(self.device)
                ).detach().squeeze()
            knn_metabest[tmpvalue]=tmp
        
        # estimated combined bound for reward
        self.upper_confidence_bounds = self.mu_hat+ self.exploration_bonus
        self.meta_values=np.divide(self.meta_values,np.sqrt(args.card))
        self.knn_metabest=knn_metabest[max(knn_metabest.keys())]
        self.bandit.knn_metabestValue =self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.knn_metabest)
        self.bandit.best_rewards_oracle =max( max(np.max(self.bandit.rewards, axis=1)),self.bandit.knn_metabestValue)
        self.best_rewards_oracle=max(max(self.best_rewards_oracle,self.bandit.best_rewards_oracle),self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.knn_metabest))
    def update_A_inv(self):
        self.A_inv[self.action] = inv_sherman_morrison(
            self.grad_approx[self.action],
            self.A_inv[self.action]
        )
        
    def run(self):
        """Run an episode of bandit.
        """
        postfix = {
            'total regret': 0.0,
            '% optimal arm': 0.0,
        }
        step=0
        # env2 = Normalizedenv2(gym.make(args2.env2))
        # env2 = gym.make(args2.env2)
        env2 = myenv2()
        #################################### Our Code ##############################
        args2.low = env2.action_space.low
        args2.high = env2.action_space.high
        #################################### Our Code ##############################

        if args2.seed > 0:
            np.random.seed(args2.seed)
            env2.seed(args2.seed)

        nb_states =args.solDim
        nb_actions =args.solDim


        agent = WOLPAgent(nb_states, nb_actions, args2)

        num_iterations=args2.train_iter
        validate_steps=args2.validate_steps
        output= args2.output
        max_episode_length=args2.max_episode_length
        debug=args2.debug
        agent.is_training = True
        step = episode = episode_steps = 0
        episode_reward = 0.
        observation = None
        import numpy as np
        import torch
        import copy
        eps=0.03
        jitter=0.1
        beta=0.2
        rho=0.1
        b=200
        N=100
        K=20
        L=1000000
        constraintMean=0.2
        g2aactionvalues=np.zeros(50)
        wolpertingeractionvalues=np.zeros(50)
        def PPOobj(uNext,u):
            obj=0.
            for i in range(K):
                for j in range(args.solDim):
                    if actions[i][j]==1:
                        obj+=(uNext[j]/u[j])*(b-rewardsCEM[i])
                    else:
                        obj+=((1-uNext[j])/(1-u[j]))*(b-rewardsCEM[i])
                    obj-=(beta*(u[j]*torch.log(u[j]/uNext[j])))
                    obj-=(beta*(u[j]*torch.log(u[j]/uNext[j])))
            return obj

        actions=np.zeros([K,args.solDim])
        rewardsCEM=np.zeros(K)
        actionsN=np.zeros([N,args.solDim])
        rewardsN=np.zeros(N)
        constraintN=np.zeros(N)
        actionhistoryBest=np.zeros([5,args.solDim])
        actionhistoryWorst=np.zeros([5,args.solDim])
        rewardhistoryBest=np.zeros(5)
        rewardhistoryWorst=np.zeros(5)
        u=np.random.random([args.solDim])
        uNext=u.copy()
        action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] for i in range(args.solDim)]
        reward=sum(action)-20*checkFea(action)
        actions[0]=action
        def feedback(x,m,constraintMean):
            return m.forward(torch.FloatTensor(x)).detach().squeeze()-10*constraintMean*checkFea(x)
        rewardsCEM[0]=feedback(action,self.model,constraintMean)
        actionsN[0]=action
        rewardsN[0]=feedback(action,self.model,constraintMean)
        actionhistoryBest[0]=action
        actionhistoryWorst[0]=action
        rewardhistoryBest[0]=feedback(action,self.model,constraintMean)
        rewardhistoryWorst[0]=feedback(action,self.model,constraintMean)
        rnd=0
        iteration=0
        
        actor=Actorg2a(args.solDim,args3)
        critic=Criticg2a(args.solDim)
        criterion = nn.MSELoss()
        opt1 =torch.optim.Adam(actor.parameters(), lr=1e-3)
        opt2 =torch.optim.Adam(critic.parameters(), lr=1e-3)
        iteration=0
        history=np.ones(args.solDim)
        constraint=0
        calactionhistoryBest=[]
        
        self.elite=[]
        with tqdm(total=self.bandit.T, postfix=postfix) as pbar:
            for t in range(self.bandit.T):
                # update confidence of all arms based on observed features at time t
                self.update_confidence_bounds()
                # pick action with the highest boosted estimated reward
                self.action = self.sample_action()
                self.actions[t] = self.action
                # update approximator
                if t %2==0:# self.train_every == 0:
                    self.train()
                # update exploration indicator A_inv
                self.update_A_inv()
                #print(t,self.action,self.bandit.best_rewards_oracle[t],self.bandit.rewards[t, self.action])
                print('hhhhh',self.bandit.hwithoutconstraints(self.bandit.features[self.iteration][self.action]*np.sqrt(args.card))-1*checkFea(torch.from_numpy(self.bandit.features[self.iteration][self.action])*np.sqrt(args.card)),checkFea(torch.from_numpy(self.bandit.features[self.iteration][self.action])*np.sqrt(args.card)))
                if self.best_recommended_reward<self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.bandit.features[self.iteration][self.action]):
                    self.best_recommended_reward=self.bandit.hwithoutconstraints(np.sqrt(args.card)*self.bandit.features[self.iteration][self.action])
                    self.best_recommended_action=self.bandit.features[self.iteration][self.action]
                self.best_sample_rewards_list.append( self.best_rewards_oracle)
                self.rewards_list.append(self.bandit.hwithoutconstraints(self.bandit.features[self.iteration][self.action]*np.sqrt(args.card)))
                self.feasi.append(checkFea(self.bandit.features[self.iteration][self.action]*np.sqrt(args.card)))
                # increment counter
                self.iteration += 1
                
                
                # log
                # log
                n_optimal_arm = np.sum(
                    self.actions[:self.iteration]==self.bandit.best_actions_oracle[:self.iteration]
                )
                postfix['% optimal arm'] = '{:.2%}'.format(n_optimal_arm / self.iteration)
                
                if t % self.throttle == 0:
                    pbar.set_postfix(postfix)
                    pbar.update(self.throttle)
                if t>=100 and t%20==0:
                    self.elite=[]
                    if t==100:
                        MM=100
                    else:
                        MM=20
                    for j in range(MM):
                        
                        iteration+=1
                        if iteration<100:
                            action=[1.]*args.card+[0]*(args.solDim-args.card)
                            np.random.shuffle(action)
                            xs=[action]
                            action=np.array(action)
                            ys=[self.model.forward(torch.FloatTensor(action)).detach().squeeze()-constraint*checkFea(action)]
                            if j>=18 and checkFea(action)<0.3:
                                self.elite.append(np.divide(action,np.sqrt(args.card)))
                        else:
                            action=actor(1)

                            if iteration==60:
                                constraint=10#100*sum(rewards)/len(rewards)
                                std=np.array(rewards).std()
                            if iteration>1 and iteration%60==0:
                                constraint=min(100*sum(rewards)/len(rewards),max(0,constraint + 0.005*(-0.0+sum(constraints)/len(constraints))))
                            done=1
                            action2=[action[i]+0.0002*np.sqrt(np.log(iteration)/history[i]) for i in range(args.solDim)]
                            action2=np.array([1 if i in np.argsort(action2)[-args.card:] else 0  for i in range(args.solDim)])
                            history+=np.array(action2)
                            loss=-critic(action)
                            opt1.zero_grad()
                            loss.backward()
                            opt1.step()
                            if j>=18 and checkFea(action2)<0.3:
                                self.elite.append(np.divide(action2,np.sqrt(args.card)))
                            xs=[action]
                            ys=[self.model.forward(torch.FloatTensor(action2)).detach().squeeze()-constraint*checkFea(action2)]
                            g2aactionvalues[(iteration-100)%50]=ys[0]
                        if iteration>0:
                            loss2=criterion(critic(torch.tensor(xs[-1])),torch.tensor(ys[-1]).float())
                            opt2.zero_grad()
                            loss2.backward()
                            opt2.step()
                        ##g2a co-training
                        if iteration>150 and g2aactionvalues.std()<0.1:
                            for _ in range(10):
                                action=actor(1)
                                loss=0
                                for i in range(args.solDim):
                                    loss-=(self.best_recommended_action[i]*torch.log(action[i])+(1-self.best_recommended_action[i])*torch.log(1-action[i]))
                                loss/=args.solDim
                                opt1.zero_grad()
                                loss.backward()
                                opt1.step()
                                
                        u=uNext.copy()
                        action=[np.random.choice([0,1],size=1,p=[1-u[i],u[i]])[0] if 0<=u[i]<=1 else np.random.choice([0,1],size=1,p=[1/2,1/2])[0] for i in range(args.solDim)]
                        reward=sum(action)-20*checkFea(action)
                        actions[(rnd%K)]=action
                        rewardsCEM[(rnd%K)]=feedback(action,self.model,constraintMean)
                        actionsN[(rnd%N)]=action
                        rewardsN[(rnd%N)]=feedback(action,self.model,constraintMean)
                        constraintN[(rnd%N)]=checkFea(action)
                        if reward>min(rewardhistoryBest) and checkFea(action)<0.1:
                            actionhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=action
                            rewardhistoryBest[np.argwhere(rewardhistoryBest==min(rewardhistoryBest))]=feedback(action,self.model,constraintMean)
                     #   if reward<max(rewardhistoryWorst) and feedback(action,self.model,constraintMean)<0.15:
                          #      actionhistoryWorst[np.argwhere(rewardhistoryWorst==min(rewardhistoryWorst))]=action
                           #     rewardhistoryWorst[np.argwhere(rewardhistoryWorst==min(rewardhistoryWorst))]=feedback(action,self.model,constraintMean)
                        if rnd%K==0 and rnd%N!=0:
                            b=rewardsCEM.mean()
                            uNext=torch.tensor(uNext,requires_grad=True)
                            optimizer=torch.optim.Adam([uNext],lr=2e-3)
                            for tmp in range(5):
                                pre=-PPOobj(uNext,u)
                                #pre.requires_grad=True
                                optimizer.zero_grad()
                                pre.backward()
                                optimizer.step() 
                            uNext=uNext.detach().numpy()
                            uNext=np.clip(uNext,jitter,1-jitter)
                        if rnd%N==0:
                            cnt=0
                            pointer=0
                            elite=[]
                            while cnt<int(N*rho) and pointer<N:
                                if constraintN[np.argsort(rewardsN)[pointer]]<0.3:
                                    elite.append(np.argsort(rewardsN)[pointer])              
                                    cnt+=1
                                pointer+=1
                            if cnt<int(N*rho):
                                while cnt<int(N*rho) and pointer<N:
                                    if np.argsort(rewardsN)[pointer] not in elite:
                                        elite.append(np.argsort(rewardsN)[pointer])
                                    pointer+=1
                            actionsN[elite]
                            tmp=actionsN[elite].reshape([len(elite),args.solDim]).T
                            for i in range(args.solDim):
                                uNext[i]=max(min(np.exp(-t/L)*uNext[i] +(1-np.exp(-t/L))*np.sum(tmp[i])/len(elite),1-jitter),jitter)
                        #    tmphistoryBest=actionhistoryBest.T
                         #   for i in range(args.solDim):
                          #      uNext[i]=max(min(uNext[i]/2+np.sum(tmphistoryBest[i])/len(elite)/2,1-jitter),jitter)
                            constraintMean=constraintN.mean()
                        rnd+=1  
                        
                        # reset if it is the start of episode
                        if observation is None:
                            observation = deepcopy(env2.reset())
                            agent.reset(observation)
                        if t==100:
                                action = agent.random_action()
                        else:    
                            # agent pick action ...        
                            action = agent.select_action(observation)
                        observation2, reward, done, info = env2.step(action,self.model)
                        wolpertingeractionvalues[(iteration-100)%50]=reward
                        action=observation2.copy()
                        if j>=18 and checkFea(action)<0.3:
                            self.elite.append(np.divide(action,np.sqrt(args.card)))
                            #print('gggggggg',1*(a.dot(action))**1/(np.sqrt(10))**1,checkFea(action))
                        # env2 response with next_observation, reward, terminate_info
                        if max_episode_length and episode_steps >= max_episode_length -1:
                            done = True

                        # agent observe and update policy
                        agent.observe(reward, observation2, done)
                        if step > args2.warmup:
                            agent.update_policy()

                        # [optional] evaluate
                        
                        # [optional] save intermideate model
                        if step % int(num_iterations/3) == 0:
                            agent.save_model(output)

                        # update 
                        step += 1
                        
                        episode_steps += 1
                        episode_reward += reward
                        observation = observation2.copy()
                        
                        if done: # end of episode
                            
                            agent.memory.append(
                                observation,
                                agent.select_action([list(observation)]),
                                0., False
                            )

                            # reset
                            observation = None
                            episode_steps = 0
                            episode_reward = 0.
                            episode += 1
                        ##wolpertinger co-training
                        if iteration>150 and wolpertingeractionvalues.std()<0.1:
                            for _ in range(10):
                                observation = observation2.copy()
                                action = agent.select_action2([list(observation)])
                                loss=0
                                for i in range(args.solDim):
                                    loss-=(self.best_recommended_action[i]*torch.log(action[i])+(1-self.best_recommended_action[i])*torch.log(1-action[i]))
                                loss/=args.solDim
                                agent.actor_optim.zero_grad()
                                loss.backward()
                                agent.actor_optim.step()
                    calactionhistoryBest.append(len(actionhistoryBest))
                    for i in actionhistoryBest:
                        self.elite.append(np.divide(i,np.sqrt(args.card)))
                        #print('--',1*(a.dot(action))**1/(np.sqrt(10))**1,checkFea(action))

In [7]:
import numpy as np
import itertools


class ContextualBandit():
    def __init__(self,
                 T,
                 n_arms,
                 n_features,
                 hwithoutconstraints,
                 noise_std=1.0,
                ):
        # number of rounds
         # number of rounds
        self.T = T
        # number of arms
        self.n_arms = n_arms
        # number of features for each arm
        self.n_features = n_features
        # average reward function
        # h : R^d -> R
        self.hwithoutconstraints=hwithoutconstraints
        self.knn_metabestValue=-100
        self.best_rewards_oracle = -100
        # standard deviation of Gaussian reward noise
        self.noise_std = noise_std
        # generate random features
        self.reset()
    @property
    def arms(self):
        """Return [0, ...,n_arms-1]
        """
        return range(self.n_arms)
        
    def reset(self):
        """Generate new features and new rewards.
        """
        self.reset_features()
        self.reset_rewards()

    def reset_features(self):
        """Generate normalized random N(0,1) features.
        """
        x1,x2=[],[]
        for i in range(self.T):
            tmp1,tmp2=[],[]
            for j in range(self.n_arms):
                a=np.array([0]*(self.n_features-args.card)+[1]*args.card)
                np.random.shuffle(a)
                a=np.divide(a,np.sqrt(args.card))
                tmp1.append(a)
            for j in range(self.n_features):
                a=np.array([0.0]*j+[1.0]+[0.0]*(self.n_features-j-1))
                tmp2.append(a)
            x1.append(tmp1)
            x2.append(tmp2)
        x1,x2=np.array(x1),np.array(x2)
        #x1 /= np.repeat(np.linalg.norm(x1, axis=-1, ord=2), self.n_features).reshape(self.T, self.n_arms, self.n_features)
        self.features = x1
        self.meta_features=x2
        self.cross_features=[]
        for i in range(self.n_features):
            for j in range(self.n_features):
                tmp=np.zeros(self.n_features)
                tmp[i]=1
                tmp[j]=1
                self.cross_features.append(tmp)
                

    def reset_rewards(self):
        """Generate rewards for each arm and each round,
        following the reward function h + Gaussian noise.
        """
        
        self.rewards = np.random.random([self.T, self.n_arms])
        # to be used only to compute regret, NOT by the algorithm itself
        self.best_rewards_oracle =max( max(np.max(self.rewards, axis=1)),self.knn_metabestValue)
        self.best_actions_oracle = np.argmax(self.rewards, axis=1)


In [8]:
class NeuralUCB(UCB):
    """Neural UCB.
    """
    def __init__(self,
                 bandit,
                 hidden_size=20,
                 n_layers=2,
                 reg_factor=1.0,
                 delta=0.01,
                 confidence_scaling_factor=-1.0,
                 training_window=100,
                 p=0.0,
                 learning_rate=0.01,
                 epochs=1,
                 train_every=1,
                 throttle=1,
                 use_cuda=False,
                ):

        # hidden size of the NN layers
        self.hidden_size = hidden_size
        # number of layers
        self.n_layers = n_layers
        
        # number of rewards in the training buffer
        self.training_window = training_window
        
        # NN parameters
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        self.use_cuda = use_cuda
        if self.use_cuda:
            raise Exception(
                'Not yet CUDA compatible : TODO for later (not necessary to obtain good results')
        self.device = torch.device('cuda' if torch.cuda.is_available() and self.use_cuda else 'cpu')
    
        # dropout rate
        self.p = p

        # neural network
        self.model = Model(input_size=bandit.n_features, 
                           hidden_size=self.hidden_size,
                           n_layers=self.n_layers,
                           p=self.p
                          ).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        super().__init__(bandit, 
                         reg_factor=reg_factor,
                         confidence_scaling_factor=confidence_scaling_factor,
                         delta=delta,
                         throttle=throttle,
                         train_every=train_every,
                        )

    @property
    def approximator_dim(self):
        """Sum of the dimensions of all trainable layers in the network.
        """
        return sum(w.numel() for w in self.model.parameters() if w.requires_grad)
    
    @property
    def confidence_multiplier(self):
        """Constant equal to confidence_scaling_factor
        """
        return self.confidence_scaling_factor
    
    def update_output_gradient(self):
        """Get gradient of network prediction w.r.t network weights.
        """
        for a in self.bandit.arms:
            x = torch.FloatTensor(
                self.bandit.features[self.iteration, a].reshape(1,-1)
            ).to(self.device)
            
            self.model.zero_grad()
            y = self.model(x)
            y.backward()
            
            self.grad_approx[a] = torch.cat(
                [w.grad.detach().flatten() / np.sqrt(self.hidden_size) for w in self.model.parameters() if w.requires_grad]
            ).to(self.device)
            
    def reset(self):
        """Reset the internal estimates.
        """
        self.reset_upper_confidence_bounds()
        self.reset_actions()
        self.reset_A_inv()
        self.reset_grad_approx()
        self.iteration = 0

    def train(self):
        """Train neural approximator.
        """
        iterations_so_far = range(np.max([0, self.iteration-self.training_window]), self.iteration+1)
        actions_so_far = self.actions[np.max([0, self.iteration-self.training_window]):self.iteration+1]

        x_train = torch.FloatTensor(self.bandit.features[iterations_so_far, actions_so_far]).to(self.device)
        y_train = torch.FloatTensor(self.bandit.rewards[iterations_so_far, actions_so_far]).squeeze().to(self.device)
        
        # train mode
        self.model.train()
        for _ in range(self.epochs):
            y_pred = self.model.forward(x_train).squeeze()
            loss = nn.MSELoss()(y_train, y_pred)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
        
    def predict(self):
        """Predict reward.
        """
        # eval mode
        self.model.eval()
        self.mu_hat= self.model.forward(
            torch.FloatTensor(self.bandit.features[self.iteration]).to(self.device)
        ).detach().squeeze()


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
T = int(5e3)
n_arms =10
n_features = args.solDim
noise_std = 0.5

confidence_scaling_factor = noise_std

n_sim = 1

p = 0.2
hidden_size = 4#16
epochs = 100#100
train_every = 10#10
confidence_scaling_factor = 1.0
use_cuda = False

In [ ]:
bandit = ContextualBandit(T, n_arms, n_features,hwithoutconstraints, noise_std=noise_std)

regrets = np.empty((n_sim, T))

for i in range(n_sim):
    bandit.reset_rewards()
    model = NeuralUCB(bandit,
                      hidden_size=hidden_size,
                      reg_factor=1.0,
                      delta=0.1,
                      confidence_scaling_factor=confidence_scaling_factor,
                      training_window=100,
                      p=p,
                      learning_rate=0.01,
                      epochs=epochs,
                      train_every=train_every,
                      use_cuda=use_cuda
                     )
    model.run()
    regrets[i] = np.cumsum(model.regrets)

In [ ]:
#np.save('samplingpercentage_fm.npy',model.actions)

In [ ]:
np.save('reward_mixpolicy_para10_realorderyahoo.npy',mofm.rewards_list)
np.save('constraint_mixpolicy_para10_realorderyahoo.npy',mofm.feasi)

In [ ]:
with gurobipy.Env(empty=True) as env:
    env.setParam('OutputFlag', 0)
    env.setParam('IterationLimit',600)
    env.start()
    with gurobipy.Model(env=env) as m:
        solverSol=np.divide(solver(np.array([1]*20),m),np.sqrt(args.card)) 